# Instalación:


### Las primeras 3 celdas solo se ejecutan 1 vez




In [ ]:
import os, sys
from google.colab import drive

drive.mount('/content/drive')
nb_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/DREAMONGYM/Colab Notebooks', nb_path)
sys.path.insert(0,nb_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileExistsError: ignored

## Instalación de biblioteca Stable Baseline



### Biblioteca que contiene los modelos


In [ ]:
!pip install --target=$nb_path git+https://github.com/Stable-Baselines-Team/stable-baselines3-contrib

  Cloning https://github.com/Stable-Baselines-Team/stable-baselines3-contrib to /tmp/pip-req-build-tzcqvoee
  Running command git clone --filter=blob:none --quiet https://github.com/Stable-Baselines-Team/stable-baselines3-contrib /tmp/pip-req-build-tzcqvoee
  Resolved https://github.com/Stable-Baselines-Team/stable-baselines3-contrib to commit 707cb0f9cd9104c989a3e4d4cf30965b64ca7339
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Instalación del Dream On Gym

### Contiene el ambiente de redes opticas y el simulador

In [ ]:
!pip install --target=$nb_path dream-on-gym-v2==0.0.6



---

# Codigo



In [ ]:
import os
import sys
from google.colab import drive
drive.mount('/content/drive')
sys.path.append('/content/drive/My Drive/DREAMONGYM/Colab Notebooks')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import torch as th

from dreamongymv2.simNetPy import *
from dreamongymv2.gym_basic import *
import gymnasium

sys.modules["gym"] = gymnasium

In [ ]:
bands = ['L','C','S','E']

episodeRewardAccum = 0
iteration = 0
episodeIteration = 100
episodeRewardsArray = []

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def getBand(actionBand):
    if (actionBand == 0):
        return 'C'
    elif (actionBand == 1):
        return 'L'
    elif (actionBand == 2):
        return 'S'
    elif (actionBand == 3):
        return 'E'

    return 'NoBand'

In [ ]:
def state():
    connectionEvent = env.getSimulator().connectionEvent
    route = env.getSimulator().controller.path[connectionEvent.source][connectionEvent.destination][0]
    slotsQuantity = 0
    for i in range(4):
        slotsQuantity = slotsQuantity + route[0].getSlots(getBand(i))
    obs = [0] * slotsQuantity
    slotInit = 0
    for i in range(len(route)):
        slotIterator = 0
        for j in range(4):
            slotInit = slotIterator
            for k in range(slotInit, slotInit + route[i].getSlots(getBand(j))):
                slotIterator = slotIterator + 1
                if route[i].getSlot(k-slotInit,getBand(j)):
                    obs[k] = obs[k] + 1
    return obs


In [ ]:
# def stateAllNetwork():
#     connectionEvent = env.getSimulator().connectionEvent
#     routes = env.getSimulator().controller.path[connectionEvent.source][connectionEvent.destination]
#     slotsQuantity = 0
#     for i in range(4):
#         slotsQuantity = slotsQuantity + routes[0][0].getSlots(getBand(i))
#     obs = [0] * slotsQuantity
#     slotInit = 0
#     for i in range(len(env.getSimulator().controller.network.links)):
#         slotIterator = 0
#         for z in range(4):
#             slotInit = slotIterator
#             for k in range(slotInit, slotInit + env.getSimulator().controller.network.links[i].getSlots(getBand(z))):
#                 slotIterator = slotIterator + 1
#                 if env.getSimulator().controller.network.links[i].getSlot(k-slotInit,getBand(z)):
#                     obs[k] = obs[k] + 1
#     return obs

In [ ]:
def first_fit_algorithm(src: int, dst: int, b: BitRate, c: Connection, n: Network, path, action):
    numberOfSlots = b.getNumberofSlots(0)
    link_ids = path[src][dst][0]
    c.bandSelected = "C"
    general_link = []
    for _ in range(n.getLink(0).getSlots(c.bandSelected)):
        general_link.append(False)

    for link in link_ids:
        link = n.getLink(link.id)
        link.bandSelected = c.bandSelected
        for slot in range(link.getSlots(c.bandSelected)):
            general_link[slot] = general_link[slot] or link.getSlot(slot,c.bandSelected)
    currentNumberSlots = 0
    currentSlotIndex = 0
    for j in range(len(general_link)):
        if not general_link[j]:
            currentNumberSlots += 1
        else:
            currentNumberSlots = 0
            currentSlotIndex = j + 1
        if currentNumberSlots == numberOfSlots:
            for k in link_ids:
                c.addLink(
                    k, fromSlot=currentSlotIndex, toSlot=currentSlotIndex+currentNumberSlots)
            return Controller.Status.Allocated, c
    return Controller.Status.Not_Allocated, c

In [ ]:
def agent_algorithm(src: int, dst: int, b: BitRate, c: Connection, n: Network, path, action):
    band = getBand(action[0])
    c.bandSelected = band
    numberOfSlots = b.getNumberofSlots(0)
    actionSpace = len(path[src][dst])
    link_ids = path[src][dst][0]
    general_link = []
    for _ in range(n.getLink(0).getSlots(band)):
        general_link.append(False)
    for link in link_ids:
        link = n.getLink(link.id)
        link.bandSelected = band
        for slot in range(link.getSlots()):
            general_link[slot] = general_link[slot] or link.getSlot(slot,band)
    currentNumberSlots = 0
    currentSlotIndex = 0
    for j in range(len(general_link)):
        if not general_link[j]:
            currentNumberSlots += 1
        else:
            currentNumberSlots = 0
            currentSlotIndex = j + 1
        if currentNumberSlots == numberOfSlots:
            for k in link_ids:
                c.addLink(
                    k, fromSlot=currentSlotIndex, toSlot=currentSlotIndex+currentNumberSlots)
            return Controller.Status.Allocated, c
    return Controller.Status.Not_Allocated, c

#Entrenamiento del modelo


In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.ppo.policies import MlpPolicy


# PPO

In [ ]:
def rewardBandSelectShorter(): ########### This one
    global episodeRewardAccum
    global episodeIteration
    global episodeRewardsArray
    global iteration

    iteration = iteration + 1
    if (iteration % episodeIteration) == 0:
        episodeRewardsArray.append(episodeRewardAccum)
        episodeRewardAccum = 0

    reward = 0
    connectionsNow = env.getSimulator().controller.connections

    value = env.getSimulator().lastConnectionIsAllocated()
    if (value.name == Controller.Status.Not_Allocated.name):
        reward -= 1.0
    elif connectionsNow[-1].bandSelected == "C" or connectionsNow[-1].bandSelected == "L" or connectionsNow[-1].bandSelected == "S":
        reward +=1.0
    elif connectionsNow[-1].bandSelected == "E":
        reward +=0.8

    episodeRewardAccum = episodeRewardAccum + reward
    return reward

In [ ]:
fileDirectory = '/content/drive/MyDrive/DREAMONGYM'

env = gymnasium.make("rlonenv-v0")

obs, info = env.reset()

#definir espacio (opciones del 0-3, 4 en total )
env.action_space = gymnasium.spaces.MultiDiscrete(np.array([4]))


#tamaño de los slots de las bandas
env.observation_space = gymnasium.spaces.MultiDiscrete([100] * 2720)

#Set funcion recompensa
env.setRewardFunc(rewardBandSelectShorter)
#Set estados
env.setStateFunc(state)
#env.setStateFunc(stateAllNetwork)
#Definir topologia y rutas
env.initEnviroment(fileDirectory + "/Topologias-rutas/Eurocore_4_bands.json", fileDirectory + "/Topologias-rutas/Eurocore_routes.json")
#Cantidad de conexiones para tener el simulador corriendo
env.getSimulator().goalConnections = 100000
env.getSimulator().setMu(1)
env.getSimulator().setLambda(1000)
env.getSimulator().setAllocator(first_fit_algorithm)
env.getSimulator().init()

/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.setRewardFunc to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.setRewardFunc` for environment variables or `env.get_wrapper_attr('setRewardFunc')` that will search the reminding wrappers.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.setStateFunc to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.setStateFunc` for environment variables or `env.get_wrapper_attr('setStateFunc')` that will search the reminding wrappers.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.initEnviroment to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.initEnviroment` for environment variables

meta: 0.95
objetivo: 0.95 covered: 0.0 epsilon: 0.01
meta: 0.2673105078629141
objetivo: 0.95 covered: 0.6826894921370859 epsilon: 0.01


/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.getSimulator to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.getSimulator` for environment variables or `env.get_wrapper_attr('getSimulator')` that will search the reminding wrappers.
  logger.warn(


In [ ]:
env.start()
env.getSimulator().setAllocator(agent_algorithm)

policy_args = dict(net_arch=5*[128], activation_fn=th.nn.ReLU)


model = PPO(MlpPolicy, env, verbose=0, tensorboard_log="./tb/PPO-DeepRMSA-v0/", policy_kwargs=policy_args, gamma=.95)

model.learn(total_timesteps=1000, log_interval=4)

modelDirectory = fileDirectory + "/models/"

model.save(modelDirectory+ "PPO-Eurocore-RW2")
#model.load('TRPO100720231300')

Nodes: 11
Links: 50
Goal Connections: 100000
Lambda: 1000
Mu: 1
+----------+----------+----------+----------+----------+----------+----------+
| progress |  arrives | blocking |  time(s) |  Wald CI |  A-C. CI |Wilson CI |
+----------+----------+----------+----------+----------+----------+----------+
|     5.0% |     5000 | 0.357329 |  0:00:03 | 0.013553 | 0.013548 | 0.013548 |
+----------+----------+----------+----------+----------+----------+----------+
|    10.0% |    10000 | 0.363364 |  0:00:05 | 0.009619 | 0.009617 | 0.009617 |
+----------+----------+----------+----------+----------+----------+----------+
|    15.0% |    15000 | 0.357976 |  0:00:09 | 0.007828 | 0.007828 | 0.007827 |
+----------+----------+----------+----------+----------+----------+----------+
|    20.0% |    20000 | 0.357182 |  0:00:14 | 0.006776 | 0.006776 | 0.006776 |
+----------+----------+----------+----------+----------+----------+----------+
|    25.0% |    25000 | 0.353986 |  0:00:17 | 0.006049 | 0.006048 |

/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.getSimulator to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.getSimulator` for environment variables or `env.get_wrapper_attr('getSimulator')` that will search the reminding wrappers.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:127: UserWarning: WARN: The obs returned by the `step()` method should be an int or np.int64, actual type: <class 'list'>
  logger.warn(f"{pre} should be an int or np.int64, actual type: {type(obs)}")
/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


In [ ]:
obs, info = env.reset()
probBloqueoArray = []
rewardsArray = []
rewardTemp = 0
NoAlloCounter = 0
ConnCounter = 1

iterationAccum = 100
_state = [0] * 2720
for i in range(100000):
    if (i % iterationAccum) == 0:
        probBloqueoArray.append(NoAlloCounter/ConnCounter)
        rewardsArray.append(rewardTemp)
        rewardTemp = 0
    action, _states = model.predict(_state)
    _state, _reward, terminated, truncated, info = env.step(action)
    rewardTemp = rewardTemp + _reward
    if _reward == -1:
      NoAlloCounter += 1
      ConnCounter += 1
    else:
      ConnCounter += 1

print(episodeRewardsArray)
print(rewardsArray)
print(probBloqueoArray)

[90.79999999999995, 90.99999999999996, 87.59999999999994, 84.59999999999995, 88.79999999999995, 76.39999999999996, 75.79999999999997, 86.19999999999996, 79.99999999999997, 85.59999999999995, 86.59999999999995, 84.19999999999996, 76.59999999999994, 91.39999999999995, 83.59999999999995, 82.79999999999997, 90.39999999999996, 82.59999999999997, 89.79999999999994, 82.59999999999997, 80.99999999999994, 85.59999999999998, 89.39999999999996, 80.19999999999997, 80.39999999999995, 78.39999999999998, 85.19999999999996, 87.59999999999997, 80.19999999999997, 83.79999999999997, 89.79999999999995, 84.59999999999998, 85.99999999999997, 83.99999999999996, 80.99999999999996, 84.59999999999997, 88.79999999999995, 83.19999999999996, 87.99999999999996, 89.79999999999995, 85.19999999999999, 82.99999999999994, 87.59999999999997, 77.79999999999998, 81.59999999999997, 89.19999999999996, 80.19999999999996, 89.99999999999997, 86.59999999999994, 79.99999999999997, 77.39999999999999, 87.19999999999997, 78.39999999

In [ ]:
def rewardLengthBased():
    global episodeRewardAccum
    global episodeIteration
    global episodeRewardsArray
    global iteration

    iteration = iteration + 1
    if (iteration % episodeIteration) == 0:
        episodeRewardsArray.append(episodeRewardAccum)
        episodeRewardAccum = 0

    connectionsNow = env.getSimulator().controller.connections
    controllerNow = env.getSimulator().controller
    #print("Source: ", connectionEvent.source, " Destination: ", connectionEvent.destination, "Path: ", len(controllerNow.path[connectionEvent.source][connectionEvent.destination]))

    value = env.getSimulator().lastConnectionIsAllocated()

    if (value.name == Controller.Status.Not_Allocated.name):
        value = -1
    else:
        #print("Last Connection ID: ", connectionsNow[-1].id)
        #print("Last Connection Links: ", connectionsNow[-1].links)
        # src, dst y actualPath of last connection
        src = connectionsNow[-1].links[0].src
        dst = connectionsNow[-1].links[-1].dst
        actualPath = connectionsNow[-1].links
        actualPathLength = 0
        for link in actualPath:
          actualPathLength += link.length
        # Possible paths
        paths = controllerNow.path[src][dst]
        PathsLength = []
        for path in paths:
          linksCumLength = 0
          for link in path:
            linksCumLength += link.length
          PathsLength.append(linksCumLength)

        if (actualPathLength <= PathsLength[0]):
          #print("it was shortest !")
          value = 1
        elif (actualPathLength <= PathsLength[1] or actualPathLength <= PathsLength[2]):
          #print("Wasnt shortest reawrd =  0.9")
          value = 0.9
        else:
          #print("Wasnt shortest reawrd =  0.8")
          value = 0.8

        #print("Lets compare ...")
        #print("Length of Whole Connection: ", actualPathLength)
        #print("Lengths of ALL possible paths: ", PathsLength)

    episodeRewardAccum = episodeRewardAccum + value
    return value

In [ ]:
fileDirectory = '/content/drive/MyDrive/DREAMONGYM'

env = gymnasium.make("rlonenv-v0")

obs, info = env.reset()

#definir espacio (opciones del 0-3, 4 en total )
env.action_space = gymnasium.spaces.MultiDiscrete(np.array([4]))


#tamaño de los slots de las bandas
env.observation_space = gymnasium.spaces.MultiDiscrete([100] * 2720)

#Set funcion recompensa
env.setRewardFunc(rewardLengthBased)
#Set estados
env.setStateFunc(state)
#env.setStateFunc(stateAllNetwork)
#Definir topologia y rutas
env.initEnviroment(fileDirectory + "/Topologias-rutas/Eurocore_4_bands.json", fileDirectory + "/Topologias-rutas/Eurocore_routes.json")
#Cantidad de conexiones para tener el simulador corriendo
env.getSimulator().goalConnections = 100000
env.getSimulator().setMu(1)
env.getSimulator().setLambda(1000)
env.getSimulator().setAllocator(first_fit_algorithm)
env.getSimulator().init()

meta: 0.95
objetivo: 0.95 covered: 0.0 epsilon: 0.01
meta: 0.2673105078629141
objetivo: 0.95 covered: 0.6826894921370859 epsilon: 0.01


/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.setRewardFunc to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.setRewardFunc` for environment variables or `env.get_wrapper_attr('setRewardFunc')` that will search the reminding wrappers.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.setStateFunc to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.setStateFunc` for environment variables or `env.get_wrapper_attr('setStateFunc')` that will search the reminding wrappers.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.initEnviroment to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.initEnviroment` for environment variables

In [ ]:
env.start()
env.getSimulator().setAllocator(agent_algorithm)

policy_args = dict(net_arch=5*[128], activation_fn=th.nn.ReLU)


model = PPO(MlpPolicy, env, verbose=0, tensorboard_log="./tb/PPO-DeepRMSA-v0/", policy_kwargs=policy_args, gamma=.95)

model.learn(total_timesteps=1000, log_interval=4)

modelDirectory = fileDirectory + "/models/"

model.save(modelDirectory+ "PPO-Eurocore-RW3")
#model.load('TRPO100720231300')

Nodes: 11
Links: 50
Goal Connections: 100000
Lambda: 1000
Mu: 1
+----------+----------+----------+----------+----------+----------+----------+
| progress |  arrives | blocking |  time(s) |  Wald CI |  A-C. CI |Wilson CI |
+----------+----------+----------+----------+----------+----------+----------+


/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.start to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.start` for environment variables or `env.get_wrapper_attr('start')` that will search the reminding wrappers.
  logger.warn(


|     5.0% |     5000 | 0.357329 |  0:00:02 | 0.013553 | 0.013548 | 0.013548 |
+----------+----------+----------+----------+----------+----------+----------+
|    10.0% |    10000 | 0.363364 |  0:00:04 | 0.009619 | 0.009617 | 0.009617 |
+----------+----------+----------+----------+----------+----------+----------+
|    15.0% |    15000 | 0.357976 |  0:00:07 | 0.007828 | 0.007828 | 0.007827 |
+----------+----------+----------+----------+----------+----------+----------+
|    20.0% |    20000 | 0.357182 |  0:00:11 | 0.006776 | 0.006776 | 0.006776 |
+----------+----------+----------+----------+----------+----------+----------+
|    25.0% |    25000 | 0.353986 |  0:00:16 | 0.006049 | 0.006048 | 0.006048 |
+----------+----------+----------+----------+----------+----------+----------+
|    30.0% |    30000 | 0.355721 |  0:00:19 | 0.005528 | 0.005528 | 0.005527 |
+----------+----------+----------+----------+----------+----------+----------+
|    35.0% |    35000 | 0.356618 |  0:00:21 | 0.0051

In [ ]:
obs, info = env.reset()
probBloqueoArray = []
rewardsArray = []
rewardTemp = 0
NoAlloCounter = 0
ConnCounter = 1

iterationAccum = 100
_state = [0] * 2720
for i in range(100000):
    if (i % iterationAccum) == 0:
        probBloqueoArray.append(NoAlloCounter/ConnCounter)
        rewardsArray.append(rewardTemp)
        rewardTemp = 0
    action, _states = model.predict(_state)
    _state, _reward, terminated, truncated, info = env.step(action)
    rewardTemp = rewardTemp + _reward
    if _reward == -1:
      NoAlloCounter += 1
      ConnCounter += 1
    else:
      ConnCounter += 1

print(episodeRewardsArray)
print(rewardsArray)
print(probBloqueoArray)

[90.79999999999995, 90.99999999999996, 87.59999999999994, 84.59999999999995, 88.79999999999995, 76.39999999999996, 75.79999999999997, 86.19999999999996, 79.99999999999997, 85.59999999999995, 86.59999999999995, 84.19999999999996, 76.59999999999994, 91.39999999999995, 83.59999999999995, 82.79999999999997, 90.39999999999996, 82.59999999999997, 89.79999999999994, 82.59999999999997, 80.99999999999994, 85.59999999999998, 89.39999999999996, 80.19999999999997, 80.39999999999995, 78.39999999999998, 85.19999999999996, 87.59999999999997, 80.19999999999997, 83.79999999999997, 89.79999999999995, 84.59999999999998, 85.99999999999997, 83.99999999999996, 80.99999999999996, 84.59999999999997, 88.79999999999995, 83.19999999999996, 87.99999999999996, 89.79999999999995, 85.19999999999999, 82.99999999999994, 87.59999999999997, 77.79999999999998, 81.59999999999997, 89.19999999999996, 80.19999999999996, 89.99999999999997, 86.59999999999994, 79.99999999999997, 77.39999999999999, 87.19999999999997, 78.39999999

# Reward lenght NFSNET

In [ ]:
def rewardLengthBased():
    global episodeRewardAccum
    global episodeIteration
    global episodeRewardsArray
    global iteration

    iteration = iteration + 1
    if (iteration % episodeIteration) == 0:
        episodeRewardsArray.append(episodeRewardAccum)
        episodeRewardAccum = 0

    connectionsNow = env.getSimulator().controller.connections
    controllerNow = env.getSimulator().controller
    #print("Source: ", connectionEvent.source, " Destination: ", connectionEvent.destination, "Path: ", len(controllerNow.path[connectionEvent.source][connectionEvent.destination]))

    value = env.getSimulator().lastConnectionIsAllocated()

    if (value.name == Controller.Status.Not_Allocated.name):
        value = -1
    else:
        #print("Last Connection ID: ", connectionsNow[-1].id)
        #print("Last Connection Links: ", connectionsNow[-1].links)
        # src, dst y actualPath of last connection
        src = connectionsNow[-1].links[0].src
        dst = connectionsNow[-1].links[-1].dst
        actualPath = connectionsNow[-1].links
        actualPathLength = 0
        for link in actualPath:
          actualPathLength += link.length
        # Possible paths
        paths = controllerNow.path[src][dst]
        PathsLength = []
        for path in paths:
          linksCumLength = 0
          for link in path:
            linksCumLength += link.length
          PathsLength.append(linksCumLength)

        if (actualPathLength <= PathsLength[0]):
          #print("it was shortest !")
          value = 1
        elif (actualPathLength <= PathsLength[1] or actualPathLength <= PathsLength[2]):
          #print("Wasnt shortest reawrd =  0.9")
          value = 0.9
        else:
          #print("Wasnt shortest reawrd =  0.8")
          value = 0.8

        #print("Lets compare ...")
        #print("Length of Whole Connection: ", actualPathLength)
        #print("Lengths of ALL possible paths: ", PathsLength)

    episodeRewardAccum = episodeRewardAccum + value
    return value

In [ ]:
fileDirectory = '/content/drive/MyDrive/DREAMONGYM'

env = gymnasium.make("rlonenv-v0")

obs, info = env.reset()

#definir espacio (opciones del 0-3, 4 en total )
env.action_space = gymnasium.spaces.MultiDiscrete(np.array([4]))


#tamaño de los slots de las bandas
env.observation_space = gymnasium.spaces.MultiDiscrete([100] * 2720)

#Set funcion recompensa
env.setRewardFunc(rewardLengthBased)
#Set estados
env.setStateFunc(state)
#env.setStateFunc(stateAllNetwork)
#Definir topologia y rutas
env.initEnviroment(fileDirectory + "/Topologias-rutas/NSFNet_4_bands.json", fileDirectory + "/Topologias-rutas/NSFNet_routes.json")
#Cantidad de conexiones para tener el simulador corriendo
env.getSimulator().goalConnections = 100000
env.getSimulator().setMu(1)
env.getSimulator().setLambda(1000)
env.getSimulator().setAllocator(first_fit_algorithm)
env.getSimulator().init()

meta: 0.95
objetivo: 0.95 covered: 0.0 epsilon: 0.01
meta: 0.2673105078629141
objetivo: 0.95 covered: 0.6826894921370859 epsilon: 0.01


/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.setRewardFunc to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.setRewardFunc` for environment variables or `env.get_wrapper_attr('setRewardFunc')` that will search the reminding wrappers.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.setStateFunc to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.setStateFunc` for environment variables or `env.get_wrapper_attr('setStateFunc')` that will search the reminding wrappers.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.initEnviroment to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.initEnviroment` for environment variables

In [ ]:
env.start()
env.getSimulator().setAllocator(agent_algorithm)

policy_args = dict(net_arch=5*[128], activation_fn=th.nn.ReLU)


model = PPO(MlpPolicy, env, verbose=0, tensorboard_log="./tb/PPO-DeepRMSA-v0/", policy_kwargs=policy_args, gamma=.95)

model.learn(total_timesteps=1000, log_interval=4)

modelDirectory = fileDirectory + "/models/"

model.save(modelDirectory+ "PPO-NSFNet-RW3")
#model.load('TRPO100720231300')

/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.start to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.start` for environment variables or `env.get_wrapper_attr('start')` that will search the reminding wrappers.
  logger.warn(


Nodes: 14
Links: 44
Goal Connections: 100000
Lambda: 1000
Mu: 1
+----------+----------+----------+----------+----------+----------+----------+
| progress |  arrives | blocking |  time(s) |  Wald CI |  A-C. CI |Wilson CI |
+----------+----------+----------+----------+----------+----------+----------+
|     5.0% |     5000 | 0.418316 |  0:00:02 | 0.013951 | 0.013945 | 0.013945 |
+----------+----------+----------+----------+----------+----------+----------+
|    10.0% |    10000 | 0.425957 |  0:00:05 | 0.009889 | 0.009887 | 0.009887 |
+----------+----------+----------+----------+----------+----------+----------+
|    15.0% |    15000 | 0.426772 |  0:00:09 | 0.008077 | 0.008076 | 0.008076 |
+----------+----------+----------+----------+----------+----------+----------+
|    20.0% |    20000 | 0.424179 |  0:00:14 | 0.006989 | 0.006988 | 0.006988 |
+----------+----------+----------+----------+----------+----------+----------+
|    25.0% |    25000 | 0.422743 |  0:00:17 | 0.006248 | 0.006248 |

/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.getSimulator to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.getSimulator` for environment variables or `env.get_wrapper_attr('getSimulator')` that will search the reminding wrappers.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:127: UserWarning: WARN: The obs returned by the `step()` method should be an int or np.int64, actual type: <class 'list'>
  logger.warn(f"{pre} should be an int or np.int64, actual type: {type(obs)}")
/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


In [ ]:
obs, info = env.reset()
probBloqueoArray = []
rewardsArray = []
rewardTemp = 0
NoAlloCounter = 0
ConnCounter = 1

iterationAccum = 100
_state = [0] * 2720
for i in range(100000):
    if (i % iterationAccum) == 0:
        probBloqueoArray.append(NoAlloCounter/ConnCounter)
        rewardsArray.append(rewardTemp)
        rewardTemp = 0
    action, _states = model.predict(_state)
    _state, _reward, terminated, truncated, info = env.step(action)
    rewardTemp = rewardTemp + _reward
    if _reward == -1:
      NoAlloCounter += 1
      ConnCounter += 1
    else:
      ConnCounter += 1

print(episodeRewardsArray)
print(rewardsArray)
print(probBloqueoArray)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[85, 92, 90, 90, 90, 78, 88, 74, 74, 82, 80, 82, 84, 78, 82, 84, 92, 76, 80, 86, 86, 82, 86, 82, 76, 80, 86, 70, 86, 86, 80, 82, 90, 84, 90, 86, 90, 92, 82, 86, 84, 80, 92, 84, 90, 80, 84, 80, 82, 78, 74, 80, 82, 90, 88, 80, 76, 86, 92, 86, 90, 86, 94, 80, 82, 84, 74, 82, 82, 76, 84, 80, 84, 78, 72, 78, 82, 88, 76, 86, 80, 92, 78, 78, 76, 86, 86, 88, 72, 78, 84, 76, 82, 88, 88, 78, 86, 90, 84, 78, 82, 84, 90, 86, 88, 92, 82, 96, 78, 86, 88, 82, 80, 84, 80, 88, 88, 88, 90, 84, 76, 86, 92, 76, 84, 84, 80, 78, 78, 88, 88, 88, 84, 82, 86, 86, 84, 76, 82, 80, 86, 78, 82, 84, 78, 76, 84, 80, 90, 80, 84, 82, 84, 82, 88, 84, 90, 86, 86, 84, 84, 84, 94, 82, 76, 88, 92, 94, 88, 92, 90, 90, 84, 76, 82, 88, 82, 72, 88, 86, 80, 76, 86, 90, 84, 80, 82, 74, 78, 74, 86, 80, 84, 78, 82, 86, 84, 74, 80, 84, 86, 72, 88, 88, 86, 74, 84, 78, 92, 86, 88, 84, 80, 78, 78, 82, 88, 78, 82, 80, 84, 86, 84, 90, 88, 78, 78, 86, 80, 80, 82, 76, 88, 80, 88, 80, 88, 80, 90, 90, 86, 82, 72, 86, 82, 84, 84, 88, 84, 84,